In [1]:
from competition_toolkit.dataloader import download_dataset
from torch.utils.data import Dataset
from yaml import load, dump, Loader, Dumper
import argparse
from train import create_dataloader

/home/lixado/IKT452_computer_vision/MapAI-Competition-Private/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Load dataset

In [2]:
trainDataset = download_dataset("train", 1, get_dataset=True)
testDataset = download_dataset("validation", 1, get_dataset=True)

print(trainDataset)
print(f'Sample: {trainDataset[0]}')
print(testDataset)
print(f'Sample: {testDataset[0]}')

Found cached dataset mapai_training_data (/home/lixado/.cache/huggingface/datasets/sjyhne___mapai_training_data/building_segmentation/1.0.0/b0b52f8c47ddbeae1962ab524cabb5fbed58d91cc70f9ac4c5981c071ad5f248)
Found cached dataset mapai_training_data (/home/lixado/.cache/huggingface/datasets/sjyhne___mapai_training_data/building_segmentation/1.0.0/b0b52f8c47ddbeae1962ab524cabb5fbed58d91cc70f9ac4c5981c071ad5f248)


Dataset({
    features: ['image', 'lidar', 'mask'],
    num_rows: 7000
})
Sample: {'image': '../../data/train/images/6179_495_44.tif', 'lidar': '../../data/train/lidar/6179_495_44.tif', 'mask': '../../data/train/masks/6179_495_44.tif'}
Dataset({
    features: ['image', 'lidar', 'mask'],
    num_rows: 1500
})
Sample: {'image': '../../data/validation/images/6305_471_68.tif', 'lidar': '../../data/validation/lidar/6305_471_68.tif', 'mask': '../../data/validation/masks/6305_471_68.tif'}


# Dataloader

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

opts = {
    "epochs":10,
    "lr": 1e-3,
    "config": "config/data.yaml",
    "device": "cpu",
    "task": 1,
    "data_ratio": 1.0,
    "imagesize": 500,
    "task1": {
        "batchsize": 8,
        "shuffle": True
    }
}


trainloader = iter(create_dataloader(opts, "train"))

batch = next(trainloader)
image, label, filename = batch
image = image.to(opts["device"])
label = label.to(opts["device"])
print(image.shape)
print(label.shape)
    

Found cached dataset mapai_training_data (/home/lixado/.cache/huggingface/datasets/sjyhne___mapai_training_data/building_segmentation/1.0.0/b0b52f8c47ddbeae1962ab524cabb5fbed58d91cc70f9ac4c5981c071ad5f248)



Using number of images in traindataset: 7000/7000
torch.Size([8, 3, 500, 500])
torch.Size([8, 500, 500])


# Encoder

In [4]:
encoder = nn.Sequential(
            nn.Conv2d(3, 8, 3, stride=2, padding=1), # torch.Size([128, 8, 32, 32])
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=1),
            nn.Conv2d(8, 12, 2, stride=1, padding=1), 
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=1),
            nn.Conv2d(12, 16, 2, stride=1, padding=1), 
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=1),
            nn.Conv2d(16, 32, 3, stride=2, padding=1), # torch.Size([128, 32, 16, 16])
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=1),
            nn.Conv2d(32, 64, 7),  # torch.Size([128, 64, 9, 9])
            nn.BatchNorm2d(64),
        )

encoded = encoder(image)


In [5]:
encoded.shape

torch.Size([8, 64, 118, 118])

In [8]:
decoder = nn.Sequential(
            nn.ConvTranspose2d(64, 32, 7), # torch.Size([128, 32, 15, 15])
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 4, stride=2, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 12, 2, stride=2),
            nn.BatchNorm2d(12),
            nn.ReLU(),
            nn.ConvTranspose2d(12, 8, 4, stride=1),
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.ConvTranspose2d(8, 1, 2, stride=1), # torch.Size([128, 3, 64, 64])
            nn.Tanh()
        ) 

decoded = decoder(encoded)
decoded.shape

torch.Size([8, 1, 500, 500])

In [13]:
new = torch.squeeze(decoded, dim=1)

new.shape

torch.Size([8, 500, 500])

In [4]:
x = nn.Conv2d(3, 8, kernel_size=(3,3), stride=2)(image)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")
x = nn.ReLU(True)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")

x.shape

Mean: -0.0624576210975647, Max: 0.8567994832992554, Min: -0.7377985715866089
Mean: 0.06282700598239899, Max: 0.8567994832992554, Min: 0.0


torch.Size([8, 8, 249, 249])

In [5]:
x = nn.Conv2d(8, 16, kernel_size=(3,3), stride=1)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")
x = nn.ReLU(True)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")

x.shape

Mean: -0.020735396072268486, Max: 0.3811044991016388, Min: -0.3278544247150421
Mean: 0.0362386479973793, Max: 0.3811044991016388, Min: 0.0


torch.Size([8, 16, 247, 247])

In [6]:
x = nn.Conv2d(16, 32, kernel_size=(5,5), stride=1)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")
x = nn.ReLU(True)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")

x.shape

Mean: -0.0015695798210799694, Max: 0.1335599720478058, Min: -0.11457626521587372
Mean: 0.016312049701809883, Max: 0.1335599720478058, Min: 0.0


torch.Size([8, 32, 243, 243])

In [7]:
x = nn.Conv2d(32, 64, kernel_size=(5,5), stride=1)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")
x = nn.ReLU(True)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")

x.shape

Mean: 0.0006744005950167775, Max: 0.10606221109628677, Min: -0.07724209874868393
Mean: 0.012780619785189629, Max: 0.10606221109628677, Min: 0.0


torch.Size([8, 64, 239, 239])

In [8]:
x = nn.Conv2d(64, 64, kernel_size=(5,5), stride=2)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")
x = nn.ReLU(True)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")

x.shape

Mean: 0.0008399624493904412, Max: 0.04235048219561577, Min: -0.051030874252319336
Mean: 0.008168298751115799, Max: 0.04235048219561577, Min: 0.0


torch.Size([8, 64, 118, 118])

In [9]:
x = nn.Conv2d(64, 64, kernel_size=(5,5), stride=2)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")
x = nn.ReLU(True)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")

print(x.shape)

x = nn.Flatten()(x)

print(x.shape)

Mean: 0.004215300548821688, Max: 0.037832312285900116, Min: -0.03804945945739746
Mean: 0.009681029245257378, Max: 0.037832312285900116, Min: 0.0
torch.Size([8, 64, 57, 57])
torch.Size([8, 207936])


In [10]:
x = nn.Linear(64*57*57, 57*57)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")
x = nn.ReLU(True)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")

x.shape

Mean: 0.00017022066458594054, Max: 0.036702558398246765, Min: -0.032442498952150345
Mean: 0.003443444613367319, Max: 0.036702558398246765, Min: 0.0


torch.Size([8, 3249])

# Decoder

In [11]:
x = nn.Linear(57*57, 64*57*57)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")

x.shape

Mean: 2.619838051032275e-05, Max: 0.03162980452179909, Min: -0.030906712636351585


torch.Size([8, 207936])

In [12]:
x = nn.ReLU(True)(x)
print(f"Mean: {torch.mean(x)}, Max: {torch.max(x)}, Min: {torch.min(x)}")

Mean: 0.00456753745675087, Max: 0.03162980452179909, Min: 0.0


In [ ]:
x = nn.ConvTranspose2d(2)